In [1]:
import numpy as np
import pandas as pd
import datetime
import time
import itertools
import json
import urllib
import double_dip
import re
from IPython.display import Markdown, display

# ↘ Start here! ↙


### Parameters:
- `days_from_now` -- an integer value between 0 and 4. This will dictate which showtimes are retrieved. ***NOTE:*** this is now deprecated due to Kimonolabs' changes to their freemium model.
- `ads_mins` -- an integer value indicating how many minutes of ads you think are run before each movie. My experience with Cinestar says that ~ 10 minutes of ads is what you'll experience.
- `trailers_mins` -- an integer value indicating how many minutes of trailers you think will be run before each movie. This varies between Movies and Cinemas... I tried to set the default to something reasonable (10 minutes).
- `max_waiting_mins` -- an integer value indicating how long you're willing to wait between the (estimated) end of a film and and the start time of the next film. 
- `acceptable_overlap_mins` -- an integer value indicating how much overlap between the end of a film and the start of the next film is 'acceptable'. Set this value to something above 0 when you're willing to miss the last bit of a  film, or the start of another film.
- `movies_to_exclude` -- A string or list of strings giving partial matches of titles of movies you're _NOT_ interesting in seeing. E.g., `['Alles', 'Mars']` will partially match to _Alles Steht Kopf_ and _Der Marsianer - Rettet Mark Watney_, and thus both films will be excluded from any double dip considerations.
- `interesting_movies`-- A string or list of strings giving partial matches of titles of movies you _ARE_ interested in seeing.
- `all_interesting_movies_must_be_in_dip` -- a boolean indicating how calculated double dips should be filtered, in the presence of `interesting_movies`. E.g. if
    ```
    interesting_movies = ['Spec', 'Der']
    ```
    and `all_interesting_movies_must_be_in_dip=True`, then only dips that include matches for "Spec" and "Der" will be included. If `all_interesting_movies_must_be_in_dip=False` included dips only have to match at least one of "Spec", or "Der". 
    
### Notes
- Once you've tweaked your parameters, just goto the `Cell` menu and select `Run All`
- If you need to _re-run_ the program, ***do not*** re-`Run All`. Instead:
    1. re-run the cell with all the parameters
    2. goto the cell with `double_dip.find_all_double_dips` (the cell above the 'Double Dips' section), and re-run that
  
  If you don't do this, you'll have to wait around will the program REscrapes showtime information, which is needlessly redundant.


In [2]:
days_from_now = 0 # deprecated since Kimono stopped allowing ondemand APIs
ads_mins=10
trailers_mins=10
max_waiting_mins=45
acceptable_overlap_mins=0
movies_to_exclude=['Spectre'] #set to None to disable
interesting_movies=['Bridge of Spies'] # set to None to disable
all_interesting_movies_must_be_in_dip=False # consider any dip that has at least one of the interesting movies

# ↗ Start here! ↖

<br/>
_Now go to `Cell`, and click `Run All`..._


In [3]:
# hit kimono-made Cinestar showtimes API
api_endpoint = 'http://www.kimonolabs.com/api/9q0fwoh2?apikey=2GuGNJHoWhVjhlu4yTXWyyTi8a1o8ybM&kimmodify=1'
api_endpoint = api_endpoint + '&date=' + str(days_from_now) # the data parameter is currently inoperative
results = json.load(urllib.urlopen(api_endpoint))

In [4]:
if not results['results']:
    display(Markdown('## _Error:_ Either there are no showings for your day, or you\'ve hit the API too many times...'+\
                     ' Try changing `days_from_now` and re-running?...'))
    print results

kino_program = double_dip.make_showtimes(results,
                                         ads_mins=ads_mins,
                                         trailers_mins=trailers_mins)

In [5]:
double_dips = \
double_dip.find_all_double_dips(kino_program,
                                max_waiting_mins=max_waiting_mins,
                                acceptable_overlap_mins=acceptable_overlap_mins,
                                movies_to_exclude=movies_to_exclude,
                                interesting_movies=interesting_movies,
                                all_interesting_movies_must_be_in_dip=all_interesting_movies_must_be_in_dip)

# Double Dips:

In [6]:
tmp = [display(Markdown('- ' + dip)) for dip in double_dips]

- Bridge of Spies - Der Unterhändler (OV) (15:45-18:27)

- Arlo & Spot (OV) (11:30-13:24) -> Carol (OV) (13:35-15:54) -> Bridge of Spies - Der Unterhändler (OV) (15:45-18:27)

- Arlo & Spot (OV) (11:30-13:24) -> Der kleine Prinz (OV) (13:30-15:37) -> Bridge of Spies - Der Unterhändler (OV) (15:45-18:27)

- Carol (OV) (13:35-15:54) -> Bridge of Spies - Der Unterhändler (OV) (15:45-18:27)

- Der kleine Prinz (OV) (11:00-13:07) -> Star Wars: Das Erwachen der Macht (OV) (13:00-15:35) -> Bridge of Spies - Der Unterhändler (OV) (15:45-18:27)

...

- Mr. Holmes (OV) (11:00-13:04) -> Star Wars: Das Erwachen der Macht (OV) (13:00-15:35) -> Bridge of Spies - Der Unterhändler (OV) (15:45-18:27)

- Die Peanuts - Der Film (OV) (11:00-12:49) -> Star Wars: Das Erwachen der Macht (OV) (13:00-15:35) -> Bridge of Spies - Der Unterhändler (OV) (15:45-18:27)

- Star Wars: Das Erwachen der Macht (OV) (11:00-13:35) -> Carol (OV) (13:35-15:54) -> Bridge of Spies - Der Unterhändler (OV) (15:45-18:27)

- Star Wars: Das Erwachen der Macht (OV) (11:00-13:35) -> Der kleine Prinz (OV) (13:30-15:37) -> Bridge of Spies - Der Unterhändler (OV) (15:45-18:27)

- Star Wars: Das Erwachen der Macht (OV) (13:00-15:35) -> Bridge of Spies - Der Unterhändler (OV) (15:45-18:27)

In [16]:
kino_date = datetime.datetime.today() + datetime.timedelta(days=days_from_now)
display(Markdown('# Program for _' + datetime.datetime.strftime(kino_date, '%A, %B %d, %Y') + '_'))
kino_program

# Program for _Friday, December 25, 2015_

,movie_end_time,movie_start_time,start_time,title
0,1900-01-01 18:27:00,1900-01-01 16:05:00,1900-01-01 15:45:00,Bridge of Spies - Der Unterhändler (OV)
1,1900-01-01 13:24:00,1900-01-01 11:50:00,1900-01-01 11:30:00,Arlo & Spot (OV)
2,1900-01-01 23:52:00,1900-01-01 21:50:00,1900-01-01 21:30:00,Im Herzen der See (OV)
3,1900-01-01 18:48:00,1900-01-01 16:20:00,1900-01-01 16:00:00,Spectre (OV)
4,1900-01-01 22:08:00,1900-01-01 19:40:00,1900-01-01 19:20:00,Spectre (OV)
5,1900-01-02 00:58:00,1900-01-01 23:20:00,1900-01-01 23:00:00,Krampus (OV)
6,1900-01-01 15:54:00,1900-01-01 13:55:00,1900-01-01 13:35:00,Carol (OV)
7,1900-01-01 21:49:00,1900-01-01 19:50:00,1900-01-01 19:30:00,Carol (OV)
8,1900-01-01 13:07:00,1900-01-01 11:20:00,1900-01-01 11:00:00,Der kleine Prinz (OV)
9,1900-01-01 15:37:00,1900-01-01 13:50:00,1900-01-01 13:30:00,Der kleine Prinz (OV)
